In [ ]:
!pip install git+https://github.com/kkrunal77/data_science_utils.git#egg=data_science_utils

  Cloning https://github.com/kkrunal77/data_science_utils.git to /tmp/pip-build-bRqUuy/data-science-utils
  Using cached https://files.pythonhosted.org/packages/cb/79/96df883cd6df0c86cb010e6f4ff790b7a30a45016a9509c94ea72c8695cd/numpy-1.17.1.zip
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-build-bRqUuy/numpy/setup.py", line 31, in <module>
        raise RuntimeError("Python version >= 3.5 required.")
    RuntimeError: Python version >= 3.5 required.
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-bRqUuy/numpy/
You are using pip version 8.1.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import imports

ModuleNotFoundError: No module named 'imports'

In [ ]:
import keras
print(keras.__version__)

2.2.5


In [ ]:
from tensorflow import keras
print(keras.__version__)

2.1.6-tf


In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.applications import xception,InceptionV3#change the architecture
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Nadam, SGD
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint,TensorBoard,EarlyStopping
import tensorflow as tf
import numpy as np
import os
# import Callback

"""config the model input dim, number of labels to classify, batch size to be processed"""

img_width,img_height = 299,299

classes = 4

batch_size = 32

# train_data_dir = '/home/hitagi/uni_pat_new/DATA/train'

# validation_data_dir = '/home/hitagi/uni_pat_new/DATA/val'

train_data_dir = '/home/eleven/krunal/kk/sweater_style/train'
validation_data_dir = '/home/eleven/krunal/kk/sweater_style/test'

txt_file = "sweater_style.txt"

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=0.3, shear_range=10, zoom_range=0.1, cval=0.0, horizontal_flip=True, vertical_flip=True)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size=(img_width, img_height), batch_size=batch_size,class_mode='categorical')

print(train_generator.class_indices)
dict_que = train_generator.class_indices

mapped_que = {}
for key,val in zip(dict_que.keys(),dict_que.values()):
        mapped_que[val] = key
        
text = open(txt_file,"w")
text.write(str(mapped_que))
text.close()


def know_samples():
    train_total_batches = train_generator.__len__()
    train_total_samples =  train_generator.samples
    val_total_batches = validation_generator.__len__()
    val_total_samples = validation_generator.samples
    train_b_size = train_total_samples/train_total_batches
    val_b_size = val_total_samples/val_total_batches
    return ("(TRAIN)a batch is produced with {} images".format(int(np.ceil(train_b_size)))),("(TRAIN)total batches are {} in count".format(train_total_batches),("(VAL)a batch is produced with {} images".format(int(np.ceil(val_b_size)))),("(VAL)total batches are {} in count".format(val_total_batches)))


def step_decay_schedule(initial_lr=None, decay_factor=None, step_size=None):
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return LearningRateScheduler(schedule)

know_samples()

# final_xception224.h5

top_weights_path = 'sweater_style.h5'
final_weights_path = 'final_sweater_style.h5'

os.mkdir("./logs_sweater_style_3")
tensorboard_dir = TensorBoard(log_dir="logs/",write_images=True)
base_model = xception.Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
# base_model = InceptionV3(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(classes, activation='softmax')(x)
model = Model(base_model.input, predictions)

train_steps = train_generator.__len__()
val_steps = validation_generator.__len__()


print("training 1st stage")

for layer in base_model.layers:
    layer.trainable =  True


    
model = multi_gpu_model(model,2)
model.compile(optimizer="Nadam",loss='categorical_crossentropy', metrics=['accuracy'])
lr_sched = step_decay_schedule(initial_lr=1e-5, decay_factor=0.75, step_size=100)

callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_acc', patience=2, verbose=0),lr_sched
]

history = model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs = 18, validation_data=validation_generator, validation_steps=val_steps, callbacks=callbacks_list,verbose=1,workers=8,max_queue_size=train_steps*2)
print("training 2nd stage")

model.load_weights(top_weights_path)
for layer in model.layers[:155]:
    layer.trainable = False
    
for layer in model.layers[155:]:
    layer.trainable = True
    
model.compile(optimizer="Nadam",loss='categorical_crossentropy',metrics=['accuracy'])
lr_sched = step_decay_schedule(initial_lr=1e-7, decay_factor=0.55, step_size=100)

callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_acc', patience=2, verbose=0),lr_sched
    ]
history = model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs = 10/1, validation_data=validation_generator, validation_steps=val_steps, callbacks=callbacks_list,verbose=1,workers=8,max_queue_size=train_steps*2)
print("training 3nd stage")

model.load_weights(final_weights_path)
for layer in model.layers[:165]:
    layer.trainable = False
    
for layer in model.layers[165:]:
    layer.trainable = True
    
model.compile(optimizer="Nadam",loss='categorical_crossentropy',metrics=['accuracy'])
lr_sched = step_decay_schedule(initial_lr=1e-7, decay_factor=0.55, step_size=100)

callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_acc', patience=2, verbose=0),lr_sched
    ]
history = model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs = 10/1, validation_data=validation_generator, validation_steps=val_steps, callbacks=callbacks_list,verbose=1,workers=8,max_queue_size=train_steps*2)
print("ensembled_module retrain")

model.load_weights(final_weights_path)
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer="Nadam",loss='categorical_crossentropy',metrics=['accuracy'])
lr_sched = step_decay_schedule(initial_lr=1e-7, decay_factor=0.65, step_size=100)

callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_acc', patience=2, verbose=0),lr_sched,tensorboard_dir
    ]

history = model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs = 6/1, validation_data=validation_generator, validation_steps=val_steps, callbacks=callbacks_list,verbose=1,workers=8,max_queue_size=train_steps*2)


print("-----Completed------")
print("Note: use {} for prediction:".format(final_weights_path))


Found 1490 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
{'Cardigan': 0, 'Poncho': 1, 'Pullover': 2, 'Sweater_Vest': 3}
training 1st stage
Epoch 1/18
46/47 [============================>.] - ETA: 1s - loss: 0.5537 - acc: 0.7497
Epoch 00001: val_acc improved from -inf to 0.75125, saving model to sweater_style.h5
47/47 [==============================] - 54s 1s/step - loss: 0.5530 - acc: 0.7497 - val_loss: 0.4990 - val_acc: 0.7512
Epoch 2/18
46/47 [============================>.] - ETA: 0s - loss: 0.4855 - acc: 0.7600
Epoch 00002: val_acc improved from 0.75125 to 0.78938, saving model to sweater_style.h5
47/47 [==============================] - 31s 654ms/step - loss: 0.4852 - acc: 0.7601 - val_loss: 0.4208 - val_acc: 0.7894
Epoch 3/18
46/47 [============================>.] - ETA: 0s - loss: 0.4065 - acc: 0.8008
Epoch 00003: val_acc improved from 0.78938 to 0.86000, saving model to sweater_style.h5
47/47 [==============================] - 31s 654ms/step - loss: 0.

-----Completed------
Note: use final_sweater_style.h5 for prediction:


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('loss')
plt.show()

plt.plot(history.history['val_acc'])
plt.plot(history.history['acc'])
plt.title('acc')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [ ]:
model_path = '/home/eleven/kk/sweater_style'
model.load_weights(final_weights_path)
model.save(os.path.join(os.path.abspath(model_path), 'final_sweater_style.h5'))

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('final_sweater_style.h5')

In [ ]:
label_dict = {j:i for i,j in train_generator.class_indices.items()}

In [ ]:
label_dict

{0: 'Cardigan', 1: 'Poncho', 2: 'Pullover', 3: 'Sweater_Vest'}

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.applications.inception_v3 import preprocess_input

pred = []
actual = []


for attr in os.listdir('/home/eleven/krunal/kk/sweater_style/test'):
    for i, file in enumerate(os.listdir(f'/home/eleven/krunal/kk/sweater_style/test/{attr}/')):
        test_image = image.load_img(f'/home/eleven/krunal/kk/sweater_style/test/{attr}/'+file, target_size=(299, 299))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis=0)
        test_image = preprocess_input(test_image)
        result = model.predict(test_image)[0]
        label = label_dict[np.argmax(result)]
        pred.append(label)
        actual.append(attr)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

report = classification_report(actual, pred)
print(report)

matrix = confusion_matrix(actual, pred)
print('confusion matrix \n', matrix)

              precision    recall  f1-score   support

    Cardigan       0.93      0.94      0.94       100
      Poncho       1.00      0.93      0.96       100
    Pullover       0.89      0.93      0.91       100
Sweater_Vest       0.96      0.97      0.97       100

 avg / total       0.94      0.94      0.94       400

confusion matrix 
 [[94  0  4  2]
 [ 1 93  6  0]
 [ 5  0 93  2]
 [ 1  0  2 97]]


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(actual, pred))

0.9425


In [ ]:
import pandas as pd
def classification_report_csv(report, matrix):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        row_data = list(filter(lambda x: len(x), row_data))
        row['class'] = row_data[0]
        row['precision'] = row_data[1]
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support/total'] = float(row_data[4])
        report_data.append(row)

    df1 = pd.DataFrame(report_data)
    df2 = pd.DataFrame(matrix)
    df2.columns = list(label_dict.values())
    dataframe = pd.concat([df1, df2], axis=1)
    kk = dataframe.columns.tolist()
    return dataframe[["class"]+kk[5:] + kk[1:5]]

df = classification_report_csv(report, matrix)
print(df)
df.to_csv("confusion_matrix.csv")

          class  Cardigan  Poncho  Pullover  Sweater_Vest  f1_score precision  \
0      Cardigan        94       0         4             2      0.94      0.93   
1        Poncho         1      93         6             0      0.96      1.00   
2      Pullover         5       0        93             2      0.91      0.89   
3  Sweater_Vest         1       0         2            97      0.97      0.96   

   recall  support/total  
0    0.94          100.0  
1    0.93          100.0  
2    0.93          100.0  
3    0.97          100.0  


In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from tensorflow.keras.applications.inception_v3 import preprocess_input


for folders in glob.glob('/home/eleven/kk/sweater_style/train'):
    for k in glob.glob(folders+'/*'):
        print(k)
        plt.figure()
        plt.subplots(280, 3, figsize=(15, 35))
        for i, file in enumerate(os.listdir(f'{k}/')):
            i += 1
            test_image = image.load_img(f'{k}/'+file, target_size=(299, 299))
            test_image = image.img_to_array(test_image)
            disp = test_image.copy()
            test_image = np.expand_dims(test_image, axis=0)
            test_image = preprocess_input(test_image)
            result = model.predict(test_image)[0]
            label = label_dict[np.argmax(result)]
#                 print(i)
            plt.subplot(24, 3, i)
            plt.axis('off')
            plt.title(f'Label : {label}', wrap=True, loc='left')
            plt.imshow(disp/255.)
#         break

In [ ]:
# directory = "validation_knife_set/*"
# for folders in glob.glob('validation_knife_set/*'):
#     for k in glob.glob(folders+'/*'):
#         print(k)
        
#     print(folders)
#     for k in glob.glob(folders+'/*'):
#         print(k)
# [k for folders in glob.glob('validation_knife_set/*') for k in glob.glob(folders+'/*')]

In [ ]:
len([k for k in glob.glob(folders+'/*') for folders in glob.glob(directory)] )

280

In [ ]:
# os.listdir('validation_knife_set/steak-knife-set')

In [ ]:
import os
import time
import re
import pandas as pd
import sys
import numpy as np
import tensorflow.keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Nadam

"""" change the preprocessing, imagesize, dicts:labels, model *.h5 file """

model = "final_xception224.h5"
loaded_model = load_model(model)

# directory = "./validations/"
directory = "validation_knife_set/*"

# dicts = {0: 'Alphabet', 1: 'Animal', 2: 'Argyle', 3: 'Camouflage', 4: 'Cheque', 5: 'Chevron', 6: 'Dot_Print', 7: 'Embroidery', 8: 'Floral', 9: 'Geometric', 10: 'Graphic_Print', 11: 'Houndstooth', 12: 'Paisley', 13: 'Plaid', 14: 'Script', 15: 'Solid', 16: 'Striped', 17: 'Superhero', 18: 'Vehicle'}
#dicts = {0: 'Chesterfield', 1: 'Reclining', 2: 'Sofa-bed', 3: 'Standard'}
#dicts = {0: 'Arm_chair', 1: 'Parson_chair', 2: 'Side_chair'}

# dicts = {0: 'rectangle', 1: 'round', 2: 'square'}
dicts = {0:'carving-set', 1:'knife-block-set', 2: 'knife-set-without-block', 3: 'steak-knife-set'}
model_prediction = []
image = []
score = [] 
# for images in os.listdir(directory):
for images in [k for folders in glob.glob('validation_knife_set/*') for k in glob.glob(folders+'/*')]:
    image.append(images)
    img = tensorflow.keras.preprocessing.image.load_img(images,target_size=(299,299))
    img = tensorflow.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = tensorflow.keras.applications.inception_v3.preprocess_input(img)
    classes = loaded_model.predict(img)
    scores = max(classes[0])
    score.append(scores)
    op = dicts[np.argmax(classes)]
    model_prediction.append(op)
    data_pd = {"images":image,"predicton":model_prediction,"Scores":score}
    #print(data_pd)
    frame = pd.DataFrame(data_pd)
    frame.to_csv("validation.csv",index=None)


In [ ]:
import keras

Using TensorFlow backend.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
!pima-indians-diabetes.csv

/bin/sh: 1: pima-indians-diabetes.csv: not found
